In [ ]:
import azureml.core
print("SDK Version:", azureml.core.VERSION)

# 1) ESML - TRAIN Classification, TITANIC model, and DEPLOY with predict_proba scoring

######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
#sys.path.append(os.path.abspath("../azure-enterprise-scale-ml/esml/common/"))  # NOQA: E402
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
#p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
#sys.path.append(os.path.abspath("../azure-enterprise-scale-ml/esml/common/"))  # NOQA: E402
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
#p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

In [ ]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
import pandas as pd

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../../../settings', you should copy template settings from '../settings'
p.active_model = 10
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.inference_mode = False

unregister_all_datasets=False
if(unregister_all_datasets):
    p.unregister_all_datasets(p.ws) # For DEMO purpose

p.connect_to_lake()
p.describe()

In [ ]:
global esml_dataset_titanic

In [ ]:
def before_feature_engieering():
    global esml_dataset_titanic
    
    esml_dataset_titanic = p.DatasetByName("ds01_titanic") # Get dataset
    df_bronze = esml_dataset_titanic.InData.to_pandas_dataframe()
    return df_bronze
before_feature_engieering().head()

In [ ]:
print ("This is the Python/Pandas .parqut-representation of Bronze, from VS Code/Python. Not Databricks/PySpark .parquet")
print("Look at the column name of 'Siblings_Spouses_Aboard'")
esml_dataset_titanic.Bronze.to_pandas_dataframe()

In [ ]:
print ("This is the Databricks/PySpark .parquet representation - Another folder in datalake, partitioned parquet files that Databricks/PySpark generated")
print("Look at the column name of 'Siblings_#_Spouses_Aboard'")
esml_dataset_titanic.SilverDatabricks.to_pandas_dataframe()

## `Get Databricks GOLD`: Alternative 1 
- Use ESML property to get PATH

In [ ]:
 from azureml.core import Dataset
 gold1 = Dataset.Tabular.from_parquet_files(path = [(p.Lakestore, p.GoldPathDatabricks)])
 gold1.to_pandas_dataframe().shape

## `Get Databricks GOLD`: Alternative 2
- Use ESML property with Azure ML Dataset

In [ ]:
p.GoldDatabricks.to_pandas_dataframe().shape

## Now we want to `SPLIT Databricks GOLD`, and `Train` our model.
## Q: Why? Benefits: "Best of both worlds"
### Scenario 01: In the `R&D phase` we worked in Databricks to featuer engineer our data - NOW we want to TRAIN a model with `Azure ML (CPU clusters) & ESML` - 3 lines of code & AutoML
### Scenario 02: In the `Production phase`: We want to use Databricks to featuer engineer our data - But TRAIN a model with  `Azure ML (CPU clusters) & ESML` - easy deployment/Pipeline-generation and lineage

### A) Databricks/Spark clusters, with PySpark to `crunch a lot data, very fast`
### B) Azure ML CPU clusters, with Python to get the below benefits:
- `Azure ML Datasets` with Datadrift, Lineage, Versioning/Timetravel
- `ESML MLOps & Scoring drift`
- `ESML TEST_SET calculation`
- `ESML PipelineFactory (Azure ML pipeline)`
- `ESML 1-line deploy model on AKS for ONLINE webservice`
- `ESML Datalake design for INFERENCE/SCORING`
- `Azure ML Studio: Deploy model on AKS online, or as a BATCH pipeline`
- `Azure ML / AutoML` compatibility
- `Python` compatibility (PySpark is sensitive,e.g. if it does not support your favourite libraries to train ML models)

In [ ]:
def get_databricks_gold_and_split_it_as_AzureMLDatasets():
    label = p.active_model["label"]
    train_6, validate_set_2, test_set_2 = p.split_gold_dbx(0.6,label) 

    return p.GoldDatabricks,train_6, validate_set_2, test_set_2

In [ ]:
datastore = None
gold = None
gold_train = None
try:
    datastore = p.connect_to_lake()
    gold = p.GoldDatabricks
    gold_train = p.GoldTrain
    gold_train.name
    print("Not 1st time. We have data mapped already. Now connected to LAKE")
except: # If 1st time....no Gold exists, nor any mapping
    print("1st time. Lets init, map what data we have in LAKE, as Azure ML Datasets")
    datastore = p.init() # 3) Automapping from datalake to Azure ML datasets
    gold,train_6, validate_set_2, test_set_2 = get_databricks_gold_and_split_it_as_AzureMLDatasets()

# `Lets SPLIT the Databricks GOLD: Train,Validate, Test`
- To update from the Databricks GOLD, is the basis of the TRAIN, VALIDATE, TEST

In [ ]:
gold,train_6, validate_set_2, test_set_2 = get_databricks_gold_and_split_it_as_AzureMLDatasets()

In [ ]:
gold.to_pandas_dataframe().head()

In [ ]:
p.Gold.to_pandas_dataframe().head()

In [ ]:
p.GoldDatabricks.to_pandas_dataframe().head()

#  We have now MARRIED the world of `Azure ML Datasets + Azure Databricks`

## SUMMARY - step 1
- ESML has read configuration for correct environment (DEV, TEST, PROD). 
- ESML has now `Automap` and `Autoregister` Azure ML Datasets as: `IN, SILVER, BRONZE, GOLD`
- ESML has read read `Databricks GOLD`, and splitted it to Gold_Train, Gold_Validate, Gold_Test, and auto-registered them as Azure ML Datasets
- ...We are now ready to TRAIN our model. We needed an Azure ML Dataset for that.


# 3) Production purpose: "once and only once": Wrap code
- 3 Callers: MLOps, AMLPipeline, and this notebook

In [ ]:
import repackage
repackage.add("../2_A_aml_pipeline/4_inference/batch/M10/your_code/")
from your_custom_code import M01In2GoldProcessor

#p.init()
esml_dataset1 = p.DatasetByName("ds01_titanic") # Get dataset 1
df_bronze = esml_dataset1.Bronze.to_pandas_dataframe()
silver1 = p.save_silver(esml_dataset1,df_bronze) #Bronze -> Silver

esml_dataset2 = p.DatasetByName("ds02_haircolor") # Get dataset 2
df_bronze2 = esml_dataset2.Bronze.to_pandas_dataframe()
silver2 = p.save_silver(esml_dataset2,df_bronze2) #Bronze -> Silver

df1 = M01In2GoldProcessor().M01_ds01_process_in2silver(silver1.to_pandas_dataframe())  # You can then copy this statement in your pipeline-step "in2silver_ds01...py"
df2 = M01In2GoldProcessor().M01_ds02_process_in2silver(silver2.to_pandas_dataframe())  # You can then copy this statement in your pipeline-step "in2silver_ds02...py"

merged_gold = M01In2GoldProcessor().M01_merge_silvers(df1,df2) # # You can then copy this statement in your pipeline-step "silver_merged_2_gold.py"

In [ ]:
merged_gold.head()

In [ ]:
label = p.active_model["label"]
train_6, validate_set_2, test_set_2 = p.split_gold_3(0.6,label) # Auto-register datasets in AZURE (GOLD_TRAIN | GOLD_VALIDATE | GOLD_TEST)   

In [ ]:
label

# 2) `ESML` Train model in `5 codelines`

In [ ]:
from esml import ESMLDataset, ESMLProject
from baselayer_azure_ml import AutoMLFactory,azure_metric_regression,azure_metric_classification
from azureml.train.automl import AutoMLConfig

automl_performance_config = p.get_automl_performance_config() # 1)Get config, for active environment (dev,test or prod)
aml_compute = p.get_training_aml_compute(p.ws) # 2)Get compute, for active environment

automl_config = AutoMLConfig(task = 'classification', # 4) Override the ENV config, for model(that inhertits from enterprise DEV_TEST_PROD config baseline)
                            primary_metric = azure_metric_classification.AUC, # # Note: Regression(MAPE) are not possible in AutoML
                            compute_target = aml_compute,
                            training_data = p.GoldTrain, # is 'train_6' pandas dataframe, but as an Azure ML Dataset
                            experiment_exit_score = '0.922', # DEMO purpose (0.308 for diabetes regression, 0.6 for classification titanic)
                            label_column_name = label,
                            **automl_performance_config
                        )
via_pipeline = False # Consistent/same return values from both AutoML ALTERNATIVES (run or pipeline)
best_run, fitted_model, experiment = AutoMLFactory(p).train_pipeline(automl_config) if via_pipeline else AutoMLFactory(p).train_as_run(automl_config)

- ESML has now fetched `configuration & train compute` for enterprise `environment (DEV,TEST or PROD)`
- ESML has `autogenerated` a AutoML-experiment, optinally as `pipline`, in correct environment.
- User has overridden some AutoML settings (`label, split percentage`, `target metric`), and use the `1-liner TRAIN` code snippet 

## 2b) ESML Scoring Drift/Concept Drift: Compare with `1-codeline`: Promote model or not? If better, then `Register model`
- `IF` newly trained model in `current` environment (`DEV`, `TEST` or `PROD`) scores BETTER than existing model in `target` environment, then `new model` can be registered and promoted.
- Q: Do we have `SCORING DRIFT / CONCEPT DRIFT?`
- Q: Is a model trained on NEW data better? IS the one in production degraded? (not fit for the data it scores - real world changed, other CONCEPT)
- A: - Lets check. Instead of `DataDrift`, lets look at `actual SCORING` on new data (and/or new code, feature engineering) - See if we should PROMOTE newly trained model...

In [ ]:
from baselayer_azure_ml import AutoMLFactory
target_env = p.dev_test_prod #"dev", test, prod  = Target environment. Does Model A score better than Model B?
print("SCORING DRIFT: If new model scores better in DEV (new data, or new code), we can promote this to TEST & PROD \n")

promote, m1_name, r1_id, m2_name, r2_run_id = AutoMLFactory(p).compare_scoring_current_vs_new_model(target_env)  

# To override (option 1) "definition of BETTER" - Adjust settings/model_settings.json to define & adjust "what is best in my use case"
# To override (option 2) - just Register whatever model you see is the best. "latest registered = best"
# - After, if you want to use AutoMLFactory(p).register_active_model(target_env), need to edit 'run_id' 'registered_model_version' in  /project_specifics/dev_test_prod/train/.../automl_active_model_dev.json

# To override (option 3) - inject your own LAMBDA function "your definition iof best" 
#my_def_of_what_model_is_better = lambda sklearn_model_new,sklearn_model_current : (sklearn_model_new > sklearn_model_current)
#promote, m1_name, r1_id, m2_name, r2_run_id = AutoMLFactory(p).compare_scoring_current_vs_new_model(target_env,my_def_of_what_model_is_better)

print("New Model: {} in environment {}".format(m1_name, p.dev_test_prod))
print("Existing Model: {} in environment {}".format(m2_name,target_env))

if (promote and p.dev_test_prod == target_env): # Can register a model in same workspace (test->test) - need to retrain if going from dev->test (but copy from test->prod)
    AutoMLFactory(p).register_active_model(target_env)


# TEST SET SCORING

# Test-set: Ensure we have a TEST_SET splitted

In [ ]:
label = p.active_model["label"]
try:
    p.GoldTest.name
except: 
    p.connect_to_lake()
    train_6, validate_set_2, test_set_2 = p.split_gold_3(0.6,label)

### NOW we can calcualate scoring on TEST_SET

In [ ]:
from baselayer_azure_ml import ESMLTestScoringFactory
label = p.active_model["label"]

auc,accuracy,f1, precision,recall,matrix,matthews, plt = ESMLTestScoringFactory(p).get_test_scoring_7_classification(label)

print("AUC:")
print(auc)
print()
print("Accuracy:")
print(accuracy)
print()
print("F1 Score:")
print(f1)
print()
print("Precision:")
print(precision)
print()
print("Recall:")
print(recall)
print()
print("Matchews correlation:")
print(matthews)
print()
print("Confusion Matrix:")
print(matrix)


# 3) ESML `Deploy model ONLINE` in `2 lines of code` (AKS) 
- Deploy "offline" MODEL from old `run` in environment To →  `DEV`, `TEST` or `PROD` environment
- ESML saves `API_key in Azure keyvault automatically`
- ESML auto-config solves 4 common 'errors/things': `correct compute name` and `valid replicas, valid agents, valid auto scaling`
    - Tip: You can adjust the number of replicas, and different CPU/memory configuration, or using a different compute target.

inference_config, model, best_run = p.get_active_model_inference_config(p.ws) #  Get compute power & lib-dependecies for DOCKER...for correct (Dev,Test or Prod) environment.
service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config,True) # Deploy: AKS dockerized with correct config (Dev,Test or Prod subscription & networking)

## 3b) DEPLOY TEST with ESML `2 lines of code`

X_test, y_test, tags = p.get_gold_validate_Xy() 
print(tags)
caller_id = "10965d9c-40ca-4e47-9723-5a608a32a0e4"

df = p.call_webservice(p.ws, X_test,caller_id) 
df.head()

# 3b) ESML `DEPLOY - custom scoring` file - predict proba

In [ ]:
import os
os.chdir(os.path.dirname(globals()['_dh'][0]))

my_custom_script_instead = 'scoring_file_dev_M01_titanic.py'
script_file_local = "./settings/project_specific/model/dev_test_prod/train/automl/"+my_custom_script_instead  # HERE you customize the auto-generated scoring script
script_file_abs = os.path.abspath(script_file_local)

inference_config_to_override_and_inject, model, best_run = p.get_active_model_inference_config(p.ws)
inference_config_to_override_and_inject.entry_script = script_file_abs
inference_config_to_override_and_inject.entry_script # Verify path

In [ ]:
# DEPLOY with custom InferenceConfig (custom scoring script)
service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config_to_override_and_inject, True) #2) (model,inference_config, overwrite_endpoint=True,deployment_config=None):

# INFERENCE - Scenario "Caller/Client"

## 1) Get MockData - Get some TEST-DATA via ESMLProject...the GoldTest

In [ ]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLDataset, ESMLProject

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../../../settings', you should copy template settings from '../settings'
p.inference_mode = False # We want "TRAIN" mode
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace

In [ ]:
label = p.active_model["label"]
to_score = None
try:
    X_test = p.GoldTest.to_pandas_dataframe()
    to_score = X_test.drop([label], axis=1)
    #print(to_score.head()) # gold_test_1 = Dataset.get_by_name(ws, name=p.dataset_gold_test_name_azure)
except: 
    print ("you need to have splitted GOLD dataset, GoldTest need to exist. Change next cell from MARKDOWN, to CODE, and run that. Try this again... ")
# #X_test, y_test, tags = p.get_gold_validate_Xy() # Get the X_test data, ESML knows the SPLIT and LABEL already (due to training)

## 2) Call AKS Webservice in 3 ways (A,B,C)
- A) Also let AKS save data to lake
- B) Use the ESML helper method (fetched keys from vault AND joins result + features)
- C) Simulate "Rest only" - No ESML dependency 
    - No ESML meaning: Fetch keys by your own from vault + join/format JSON yourself + save data yourself to lake)

### Alt 1 - ESML.call_webservice, `get PANDAS joined` dataframe
#### `Also saves to LAKE, automatically`

In [ ]:
#p.lakestore = p.set_lake_as_datastore(p.ws) # For AutoSave - this i NOT needed if p.init() is done...which usually is the case.
p.call_webservice(p.ws, to_score,"caller_id").head() # (X_test, firstRowOnly=True,pandas_result=True, api_uri=None,api_key="auto from keyvault")

### Alt 2 - use compute factory, control to `get JSON back` instead of PANDAS. 
#### `No saving to LAKE`

In [ ]:
result, model_version_used = p.compute_factory.call_webservice(to_score,False,False) # (X_test, firstRowOnly=True,pandas_result=True, api_uri=None,api_key="auto from keyvault")
df_res = pd.read_json(result)
to_score.join(df_res) # Need to join the FEATURES yourself, post webservice call (simulate no ESML dependancy in caller)

### Alt 3 - Simulate client witn no ESML SDK, just using the "scoring endpoint". 
- Just JSON result (No ESML dependancy `get JSON back`)
#### `No saving to LAKE` and `no JOIN` 

In [ ]:
from baselayer_azure_ml import ComputeFactory
import json
keyvault = p.ws.get_default_keyvault() # Authentica to your Azure ML workspace (ws)
api_uri = keyvault.get_secret(name='esml-dev-p02-m10-api') 
api_key = keyvault.get_secret(name='esml-dev-p02-m10-apisecret') # DEV + Titanic

#api_uri = keyvault.get_secret(name='esml-test-p02-m10-api') # TEST + Titanic
#api_key = keyvault.get_secret(name='esml-test-p02-m10-apisecret')

result_json = ComputeFactory.call_webservice_static(to_score, api_uri,api_key,firstRowOnly=False) # Simulate "REST call" (no ESML dependancy, just a wrapper for a pytnon REST call)
res_dict = json.loads(result_json.text) # json -> dictionary
df_res = pd.read_json(res_dict) # dictionary -> pandas
all_result = X_test.join(df_res) # features + result
all_result

In [ ]:
# And....you need to save the data yourself to the lake, at this location
to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
print("Save your data here, if you want to have ADF WriteBack function")
print()
print(scored_folder)
print()
print("Note: Last folder, UUID folder, should represent a 'unique scoring' for a day, but can be injected. Example: if we want a customerGUID instead ")

# END

# EXTRA - more about `AutoLake Paths`

In [ ]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLDataset, ESMLProject
p = ESMLProject() 
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace

In [ ]:
p.inference_mode = True # This flag will "change the paths", from TRAIN folder to INFERENCE folder-structure

print("")
print("INFERENCE")
print("")

for d in p.Datasets:
    print(d.Name)
    print("IN", d.InPath)
    print("Bronze", d.BronzePath)
    print("Silver", d.SilverPath)

to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path(p.date_scoring_folder)
print("Gold", to_score_folder, "  ...uuid folder, is to be able to have multiple unique scorings, same datetime")

print("")
print("TRAIN")
print("")

p.inference_mode = False # This flag will "change the paths"

for d in p.Datasets:
    print(d.Name)
    print("IN", d.InPath)
    print("Bronze", d.BronzePath)
    print("Silver", d.SilverPath)

print("Gold", p.GoldPath)